# Convert images and masks to COCO datasert

## Imports

In [ ]:
import os
import random

import IPython

import cv2

## Prepare environment

In [ ]:
os.chdir(os.path.abspath(''))
os.chdir("..")
os.getcwd()

In [ ]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')
print(nb_name)

## Constants

In [ ]:
dataset_directory = os.path.join(".", "data_in", "images", "ld_sheets")


root_directory = os.path.join(dataset_directory)
images_directory = os.path.join(root_directory, "images")
masks_directory = os.path.join(root_directory, "masks")


## Generic functions

In [ ]:
def get_mask_path(img_name):
    png_path = os.path.join(masks_directory, img_name.replace(".jpg", ".png"))
    bmp_path = os.path.join(masks_directory, img_name.replace(".jpg", ".bmp"))
    return (
        png_path
        if os.path.isfile(png_path) is True
        else bmp_path
        if os.path.isfile(bmp_path)
        else None
    )


def get_mask(img_name):
    mask_path = get_mask_path(img_name)
    if os.path.isfile(mask_path):
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE).astype(np.float32)
        mask[mask == 0.0] = 0.0
        mask[mask != 0.0] = 1.0
        return mask
    else:
        return None


def get_image(img_item):
    return {
        "image": cv2.cvtColor(
            cv2.imread(os.path.join(images_directory, img_item)),
            cv2.COLOR_BGR2RGB,
        ),
        "mask": get_mask(img_name=img_item),
    }


## Get images and masks

In [ ]:
images_filenames = list(sorted(os.listdir(images_directory)))
correct_images_filenames = [i for i in images_filenames if cv2.imread(os.path.join(images_directory, i)) is not None]

random.seed(42)
random.shuffle(correct_images_filenames)


bad_paths = []
for cif in correct_images_filenames:
    if get_mask_path(cif) is None:
        bad_paths.append(cif)

bad_paths